In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 144)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)
# model = gs.GenerativeSurrogate()
# model.load()

In [8]:
model.fit(n_steps=50, with_precisions=False, lambdac_iterations=1000)

step =  0
loss z =  6615001.0
loss z =  6614893.0
loss z =  6614787.0
loss z =  6614681.0
loss z =  6614575.0
loss z =  6614471.0
loss z =  6614364.0
loss z =  6614259.0
loss z =  6614154.5
z step =  13.507429838180542 s
loss_f =  6606282.0
loss_f =  5243915.0
thetaf step =  16.558748483657837 s
sample ==  0
loss_lambda_c =  93169718919168.0
loss_lambda_c =  92291582656512.0
loss_lambda_c =  90992808034304.0
loss_lambda_c =  89025226473472.0
sample ==  1
loss_lambda_c =  54689731182592.0
loss_lambda_c =  53691168063488.0
loss_lambda_c =  52394515759104.0
loss_lambda_c =  50368130056192.0
sample ==  2
loss_lambda_c =  819782957924352.0
loss_lambda_c =  817802038476800.0
loss_lambda_c =  814964944142336.0
loss_lambda_c =  810202362281984.0
sample ==  3
loss_lambda_c =  959225916293120.0
loss_lambda_c =  958262769876992.0
loss_lambda_c =  956837042061312.0
loss_lambda_c =  954392735907840.0
sample ==  4
loss_lambda_c =  865640592179200.0
loss_lambda_c =  865231026782208.0
loss_lambda_c = 

sample ==  15
loss_lambda_c =  19087293939712.0
loss_lambda_c =  18405950226432.0
loss_lambda_c =  17415526154240.0
loss_lambda_c =  15804928098304.0
lambdac step =  22.71327257156372 s
thetac step =  0.0046405792236328125 s
step =  3
loss z =  4694714.0
loss z =  4694610.5
loss z =  4694507.5
loss z =  4694403.5
loss z =  4694297.5
loss z =  4694190.0
loss z =  4694080.5
loss z =  4693971.5
loss z =  4693858.5
z step =  13.29863977432251 s
loss_f =  4679250.0
loss_f =  4585862.5
thetaf step =  18.674727201461792 s
sample ==  0
loss_lambda_c =  427550801920.0
loss_lambda_c =  341224751104.0
loss_lambda_c =  335899983872.0
loss_lambda_c =  333484523520.0
sample ==  1
loss_lambda_c =  225289224192.0
loss_lambda_c =  187645984768.0
loss_lambda_c =  187439775744.0
loss_lambda_c =  187422015488.0
sample ==  2
loss_lambda_c =  4222217617408.0
loss_lambda_c =  1507811917824.0
loss_lambda_c =  1392700817408.0
loss_lambda_c =  1332983234560.0
sample ==  3
loss_lambda_c =  21825341882368.0
loss_

lambdac step =  22.788851261138916 s
thetac step =  0.017880678176879883 s
step =  6
loss z =  4220181.5
loss z =  4219844.0
loss z =  4219523.0
loss z =  4219217.0
loss z =  4218923.5
loss z =  4218642.5
loss z =  4218373.0
loss z =  4218112.5
loss z =  4217861.5
z step =  14.128400802612305 s
loss_f =  4198953.0
loss_f =  4117435.5
thetaf step =  16.77570414543152 s
sample ==  0
loss_lambda_c =  329509535744.0
loss_lambda_c =  329509208064.0
loss_lambda_c =  329509142528.0
loss_lambda_c =  329509142528.0
sample ==  1
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
sample ==  2
loss_lambda_c =  1234703482880.0
loss_lambda_c =  1233572200448.0
loss_lambda_c =  1233099423744.0
loss_lambda_c =  1232815652864.0
sample ==  3
loss_lambda_c =  1641806823424.0
loss_lambda_c =  1641807085568.0
loss_lambda_c =  1641807085568.0
loss_lambda_c =  1641807085568.0
sample ==  4
loss_lambda_c =  888302534656.0
loss_lambda_

loss_f =  3766798.0
loss_f =  3689252.75
thetaf step =  17.961735248565674 s
sample ==  0
loss_lambda_c =  329509175296.0
loss_lambda_c =  329509142528.0
loss_lambda_c =  329509142528.0
loss_lambda_c =  329509142528.0
sample ==  1
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
sample ==  2
loss_lambda_c =  1231768518656.0
loss_lambda_c =  1231730638848.0
loss_lambda_c =  1231716089856.0
loss_lambda_c =  1231707570176.0
sample ==  3
loss_lambda_c =  1641806954496.0
loss_lambda_c =  1641806954496.0
loss_lambda_c =  1641806823424.0
loss_lambda_c =  1641806823424.0
sample ==  4
loss_lambda_c =  888301682688.0
loss_lambda_c =  888301682688.0
loss_lambda_c =  888301682688.0
loss_lambda_c =  888301682688.0
sample ==  5
loss_lambda_c =  1513720774656.0
loss_lambda_c =  1513720905728.0
loss_lambda_c =  1513720905728.0
loss_lambda_c =  1513720905728.0
sample ==  6
loss_lambda_c =  731135410176.0
loss_lambda_c =  731

loss_lambda_c =  187415691264.0
loss_lambda_c =  187415691264.0
sample ==  2
loss_lambda_c =  1231677030400.0
loss_lambda_c =  1231675850752.0
loss_lambda_c =  1231675457536.0
loss_lambda_c =  1231675064320.0
sample ==  3
loss_lambda_c =  1641806954496.0
loss_lambda_c =  1641807085568.0
loss_lambda_c =  1641807085568.0
loss_lambda_c =  1641806954496.0
sample ==  4
loss_lambda_c =  888301617152.0
loss_lambda_c =  888301617152.0
loss_lambda_c =  888301748224.0
loss_lambda_c =  888301748224.0
sample ==  5
loss_lambda_c =  1513720774656.0
loss_lambda_c =  1513720905728.0
loss_lambda_c =  1513720905728.0
loss_lambda_c =  1513720643584.0
sample ==  6
loss_lambda_c =  731135541248.0
loss_lambda_c =  731135475712.0
loss_lambda_c =  731135475712.0
loss_lambda_c =  731135475712.0
sample ==  7
loss_lambda_c =  458895294464.0
loss_lambda_c =  458894639104.0
loss_lambda_c =  458894376960.0
loss_lambda_c =  458894278656.0
sample ==  8
loss_lambda_c =  77439696896.0
loss_lambda_c =  77439451136.0
los

loss_lambda_c =  1641806954496.0
loss_lambda_c =  1641806954496.0
sample ==  4
loss_lambda_c =  888301682688.0
loss_lambda_c =  888301617152.0
loss_lambda_c =  888301682688.0
loss_lambda_c =  888301682688.0
sample ==  5
loss_lambda_c =  1513720643584.0
loss_lambda_c =  1513720774656.0
loss_lambda_c =  1513720774656.0
loss_lambda_c =  1513720774656.0
sample ==  6
loss_lambda_c =  731135475712.0
loss_lambda_c =  731135541248.0
loss_lambda_c =  731135541248.0
loss_lambda_c =  731135541248.0
sample ==  7
loss_lambda_c =  458894049280.0
loss_lambda_c =  458893918208.0
loss_lambda_c =  458893852672.0
loss_lambda_c =  458893819904.0
sample ==  8
loss_lambda_c =  77439418368.0
loss_lambda_c =  77439287296.0
loss_lambda_c =  77439254528.0
loss_lambda_c =  77439229952.0
sample ==  9
loss_lambda_c =  272516415488.0
loss_lambda_c =  272516415488.0
loss_lambda_c =  272516415488.0
loss_lambda_c =  272516399104.0
sample ==  10
loss_lambda_c =  661734883328.0
loss_lambda_c =  661734948864.0
loss_lambd

KeyboardInterrupt: 

In [ ]:
model.fit(n_steps=50, with_precisions=True)

In [ ]:
model.log_lambdac_mean

In [ ]:
# model = gs.GenerativeSurrogate()
# model.load()

In [9]:
test_samples = {n for n in range(0, 4)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [12]:
uf_pred, Z = model.predict(testData, z_iterations=10000)

loss_z_prediction =  185939.265625
loss_z_prediction =  179411.28125
loss_z_prediction =  173422.453125
loss_z_prediction =  167800.796875
loss_z_prediction =  162349.09375
loss_z_prediction =  157245.796875
loss_z_prediction =  152304.25
loss_z_prediction =  147399.65625
loss_z_prediction =  142581.953125
loss_z_prediction =  137696.625
loss_z_prediction =  132873.234375
loss_z_prediction =  128415.796875
loss_z_prediction =  124427.96875
loss_z_prediction =  120740.4609375
loss_z_prediction =  117421.0078125
loss_z_prediction =  114402.0390625
loss_z_prediction =  111750.796875
loss_z_prediction =  109363.8828125
loss_z_prediction =  107242.71875
loss_z_prediction =  105394.7578125
loss_z_prediction =  103762.7265625
loss_z_prediction =  102294.84375
loss_z_prediction =  100982.7578125
loss_z_prediction =  99799.734375
loss_z_prediction =  98724.5234375
loss_z_prediction =  97747.9765625
loss_z_prediction =  96846.5625
loss_z_prediction =  96016.578125
loss_z_prediction =  95248.5390

In [13]:
uf_pred[0]

array([0.        , 0.00505355, 0.01010711, ..., 1.879869  , 1.882973  ,
       1.886077  ], dtype=float32)

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes(projection='3d')

In [ ]:
ax.plot_surface(x, y, np.reshape(model.rom.interpolated_solution.array, (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[15, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
b = torch.BoolTensor([False, True, True])

In [ ]:
x = torch.tensor([[1.1, 2, 3], [4, 5, 6]])

In [ ]:
x = torch.randn(size=(256, 256))

In [ ]:
%timeit torch.dot(x.flatten(), x.flatten())

In [ ]:
b = x > .5

In [ ]:
%timeit torch.sum(x[b])

In [ ]:
x.dtype = torch.bool

In [ ]:
model.plot_input_reconstruction()

In [46]:
def plot_generated_microstructures(self):
    """
    Generates microstructures via z ~ p(z), lambdaf ~ pfNet(z)
    """
    z = torch.randn(size=(4, self.dim_z))
    lambdaf_samples = self.pfNet(z)
    
    fig, ax = plt.subplots(1, 4)
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()

    for col in range(4):
        
        im = ax[col].imshow(torch.reshape(lambdaf_samples[col],
                                                  (self.data.img_resolution,
                                                   self.data.img_resolution)).detach().numpy(),
                                    cmap='gray_r', vmin=0, vmax=1)
        ax[col].set_xticks([], [])
        ax[col].set_yticks([], [])
        ax[col].set_title('generated sample ' + str(col))

    cbar_ax = fig.add_axes([0.92, .325, 0.01, 0.34])
    fig.colorbar(im, cax=cbar_ax)
    plt.show()

In [47]:
plot_generated_microstructures(model)